In [ ]:
import numpy as np
import nltk
from keras.models import Model
from keras.layers.recurrent import LSTM
from keras.layers import Dense, Input, Embedding
from keras.preprocessing.sequence import pad_sequences
from google.colab import drive
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
WHITELIST = 'abcdefghijklmnopqrstuvwxyz1234567890?.,'

def in_white_list(_word):
  '''Check if the characters in the words are whitelisted'''
  for char in _word:
      if char in WHITELIST:
          return True

  return False 

In [ ]:
'''
Load dictionary values from  glove.twitter.27B.100d.txt file.
Add embedded values into wrd2embedding dictionary.
'''
word2embedding = None
file_path = "/content/drive/My Drive/NLP Chatbot"

def load_glove_twitter_data():
    _word2embedding = {}
    file = open(file_path+"/embedding_data/glove.twitter.27B.100d.txt", mode='rt', encoding='utf8')
    for line in file:
        words = line.strip().split()
        word = words[0]
        embeds = np.array(words[1:], dtype=np.float32)
        _word2embedding[word] = embeds        
    file.close()
    return _word2embedding
word2embedding = load_glove_twitter_data()



**Load word to index and index to word information from saved mpy files**

In [ ]:
target_word2idx = None
target_idx2word = None
max_decoder_seq_length = None
max_encoder_seq_length = None
num_decoder_tokens = None

GLOVE_EMBEDDING_SIZE = 100
HIDDEN_UNITS = 256

target_word2idx = np.load( file_path + '/word-glove-target-word2idx.npy',allow_pickle=True).item()
target_idx2word = np.load( file_path + '/word-glove-target-idx2word.npy',allow_pickle=True).item()
context = np.load( file_path + '/word-glove-context.npy',allow_pickle=True).item()

max_encoder_seq_length = context['encoder_max_seq_length']
max_decoder_seq_length = context['decoder_max_seq_length']
num_decoder_tokens = context['num_decoder_tokens']

'''
Preparing encoder inputs with defining  glove embedding size and number ond number of hudden units.
These number should be same as what we train our LSTM mode.
'''
encoder_inputs = Input(shape=(None, GLOVE_EMBEDDING_SIZE), name='encoder_inputs')
encoder_lstm = LSTM(units=HIDDEN_UNITS, return_state=True, name="encoder_lstm")
encoder_outputs, encoder_state_h, encoder_state_c = encoder_lstm(encoder_inputs)
encoder_states = [encoder_state_h, encoder_state_c]


'''
Preparing decoder inputs with defining  glove embedding size and number ond number of hudden units.
These number should be same as what we train our LSTM mode.
'''
decoder_inputs = Input(shape=(None, GLOVE_EMBEDDING_SIZE), name='decoder_inputs')
decoder_lstm = LSTM(units=HIDDEN_UNITS, return_sequences=True, return_state=True, name='decoder_lstm')
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax', name='decoder_dense')
decoder_outputs = decoder_dense(decoder_outputs)

**Prepare model object by passing encode and decoder inputs paramater.

Load train LSTM model for loading weights**

In [ ]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.load_weights( file_path + '/word-glove-weights.h5')
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
encoder_model = Model(encoder_inputs, encoder_states)

In [ ]:
decoder_state_inputs = [Input(shape=(HIDDEN_UNITS,)), Input(shape=(HIDDEN_UNITS,))]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_state_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_state_inputs, [decoder_outputs] + decoder_states)


In [ ]:
def reply(input_text):
  input_seq = []
  input_emb = []
  for word in nltk.word_tokenize(input_text.lower()):
    print(word)
    if not in_white_list(word):
      continue
    
    emb = np.zeros(shape=GLOVE_EMBEDDING_SIZE)
    if word in word2embedding:
      emb = word2embedding[word]
    input_emb.append(emb)
  
  #print(input_emb)
  input_seq.append(input_emb)
  input_seq = pad_sequences(input_seq, max_encoder_seq_length)
  states_value = encoder_model.predict(input_seq)
  target_seq = np.zeros((1, 1, GLOVE_EMBEDDING_SIZE))
  target_seq[0, 0, :] = word2embedding['start']
  target_text = ''
  target_text_len = 0
  terminated = False
  while not terminated:
    output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

    sample_token_idx = np.argmax(output_tokens[0, -1, :])
    sample_word = target_idx2word[sample_token_idx]
    target_text_len += 1

    if sample_word != 'start' and sample_word != 'end':
      target_text += ' ' + sample_word

    if sample_word == 'end' or target_text_len >= max_decoder_seq_length:
      terminated = True

    target_seq = np.zeros((1, 1, GLOVE_EMBEDDING_SIZE))
    if sample_word in word2embedding:
      target_seq[0, 0, :] = word2embedding[sample_word]
    states_value = [h, c]

  return target_text.strip()

 

In [ ]:
import nltk
nltk.download('punkt')
print(reply('Hello'))


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
hello
'' i 'm a unknown of unknown unknown unknown unknown of the unknown of unknown unknown unknown of the unknown of unknown unknown unknown of the unknown of unknown unknown unknown of the unknown of unknown unknown unknown unknown unknown unknown


In [ ]:
print(reply("Good evening"))

good
evening
i 'm sorry . i 'm going to be a unknown of unknown unknown of the unknown of unknown unknown of the unknown of unknown unknown of the unknown of unknown unknown of the unknown of unknown unknown unknown of


In [ ]:
print(reply('Jungle man fix'))

jungle
man
fix
i 'm sorry . i 'm going to be a unknown of unknown unknown . i 'm not going to be a unknown of unknown unknown of the unknown of unknown unknown of the unknown of unknown unknown unknown of
